In [340]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
! pip install cvxpy
import cvxpy as cp
from tqdm import tqdm

iris = load_iris()

7354.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [341]:
def gaussian_ker(x, y, q):
    """a function to compute the gaussian kernel of two points
    -------------------------------
    inputs : 

    x : array-like, vector
    first vector for which we want to compute the kernel

    y : array-like, vector
    second vector for which we want to compute the kernel

    q : positive float,
    value of the bandwidth of the kernel

    returns:
    ker : float,
    the value of the kernel
    -------------------------------
    """

    ker = np.exp(-q*np.linalg.norm(x-y)**2)
    return ker

In [342]:
def gram_mat(X, q):
    """
    a function to compute the gram matrix of a given dataset

    ----------------------------------------
    inputs : 
    X : array-like object, must be 2D
    the data for which we want to compute the gram matrix

    q : positive float, 
    the bandwidth of the gaussian kernel
    -----------------------------------------

    returns:
    K : the gram matrix
    """
    
    norms = np.linalg.norm(X, axis=1)**2
    dot = X@X.T
    squared_euclidian_distances = norms[:, None] - 2 * dot + norms[None, :]
    K = np.exp(-squared_euclidian_distances*q)
    return K

In [343]:
def compute_seg(x, y, nb=20):
    """
    a function used to compute the segment between two points
    -----------------------------------
    Parameters : 

    x : array-like obj,
    an input, d>=2

    y : array-like obj, 
    the second input

    nb : int, 
    the number of points we want to have between the two points

    Returns : 

    segment : array-like
    an array of shape d (dimension of x), nb
    ---------------------------------------
    """
    d = x.shape[0]
    segment = np.zeros((nb, d))
    points = np.linspace(start=0., stop=1., num=nb, endpoint=True)
    
    for i in range(nb):
        t = points[i]
        segment[i, :] = (1-t) * x + t * y
        
    return segment

In [344]:
def radius(x, sample, beta, bkb, q, ker_self=1.):
    """
    compute the radius for a given instance x
    -----------------------------------------------
    Parameters : 

    x : 1-D vector,
    the input vector

    sample : matrix, 
    the whole sample, 

    beta : array-like, 
    the calculated beta

    bkb : float,
    the result of beta.T@K@beta

    ker_self : float,
    the value of the kernel of the selected instance with itself, 
    set to 1 by default as we use mostly the gaussian kernel

    returns :

    radius : float,
    the distance between the test instance and the center of the 
    sphere enclosing all the points in the Hilbert space
    -----------------------------------------------
    """
    nb_samp = sample.shape[0]
    temp_k = np.zeros(nb_samp)

    for elem in range(nb_samp):
        temp_k[elem] = gaussian_ker(x, sample[elem], q=q)
        
    return np.sqrt(ker_self - 2*np.dot(temp_k, beta) + bkb)

Application on the Iris dataset

In [345]:
data = iris.data
target = iris.target

In [346]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Prétraitement : Normaliser les données (centrage et réduction)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Initialisation de la PCA
n_components = 3  # Choisissez le nombre de composantes principales souhaitées
pca = PCA(n_components=n_components)

# Appliquer la PCA
principal_components = pca.fit_transform(data_scaled)

# Créer un DataFrame des composantes principales
columns = [f"PC{i+1}" for i in range(n_components)]
principal_df = pd.DataFrame(data=principal_components, columns=columns)

# Afficher les composantes principales
print(principal_df)

# Variance expliquée par chaque composante
print("Variance expliquée par chaque composante :", pca.explained_variance_ratio_)

# Somme de la variance expliquée
print("Variance totale expliquée :", np.sum(pca.explained_variance_ratio_))

          PC1       PC2       PC3
0   -2.264703  0.480027  0.127706
1   -2.080961 -0.674134  0.234609
2   -2.364229 -0.341908 -0.044201
3   -2.299384 -0.597395 -0.091290
4   -2.389842  0.646835 -0.015738
..        ...       ...       ...
145  1.870503  0.386966 -0.256274
146  1.564580 -0.896687  0.026371
147  1.521170  0.269069 -0.180178
148  1.372788  1.011254 -0.933395
149  0.960656 -0.024332 -0.528249

[150 rows x 3 columns]
Variance expliquée par chaque composante : [0.72962445 0.22850762 0.03668922]
Variance totale expliquée : 0.9948212908928451


In [347]:
# Hyperparameters of the SVC procedure
N = len(principal_df)
q = 7
p = 0.7
C = 1 / (N * p)

In [348]:
gram_x = gram_mat(X=principal_df, q=q)

In [349]:
n = len(principal_df)
beta = cp.Variable(n)
gram_x += gram_x.T
gram_x /= 2
gram_x = cp.psd_wrap(gram_x)


# Formulation de l'objectif
objective = cp.Maximize(cp.sum(np.ones(n) @ beta) - cp.quad_form(beta, gram_x))

# Contraintes
constraints = [
    beta >= 0,  # 0 <= beta_j
    beta <= C,  # beta_j <= C
    cp.sum(beta) == 1  # La somme des éléments de beta doit être égale à 1
]

# Définir le problème d'optimisation
problem = cp.Problem(objective, constraints)

# Résoudre le problème
problem.solve()

np.float64(0.9798588124462496)

In [350]:
true_beta = beta.value
beta_k_beta = true_beta.T @ gram_x @ true_beta

In [351]:
index_of_sv = []

for i in range(N):
    if 1e-10 < true_beta[i] < C - 1e-3:
        index_of_sv.append(i)

print(index_of_sv) 

[0, 4, 9, 11, 16, 20, 21, 26, 28, 31, 38, 45, 46, 47, 49, 54, 55, 58, 61, 71, 73, 74, 75, 78, 80, 88, 90, 92, 94, 101, 103, 112, 116, 126, 127, 128, 133, 134, 137, 138, 139, 140, 142]


In [352]:
potential_sv = principal_df.iloc[index_of_sv, :].to_numpy()

In [353]:
print('number of potential SVs:', len(potential_sv))

number of potential SVs: 43


In [354]:
for i in range(n):
    if true_beta[i] < 1e-10:
        true_beta[i] = 0

In [329]:
gram_x = gram_mat(X=principal_df, q=q)
gram_x += gram_x.T
gram_x /= 2
r = []
beta_k_beta = true_beta.T @ gram_x @ true_beta
for i in potential_sv:
    temp_K = np.zeros(n)
    for elem in range(n):
        temp_K[elem] = gaussian_ker(i, principal_df.iloc[elem, :], q=q)
    r_xi = np.sqrt(1 - 2*np.dot(temp_K, true_beta) + beta_k_beta)
    r.append(r_xi)
rad = max(r)

In [330]:
adjacency_mat = np.zeros((n, n))

for i in tqdm(range(n)):
    for j in range(i+1, n):
        decision = True
        segment = compute_seg(x=principal_df.to_numpy()[i,:], y=principal_df.to_numpy()[j,:])
        list_of_val = []
        for point in segment:
            dist = radius(x=point, beta=true_beta, bkb=beta_k_beta, sample=principal_df.to_numpy(), q=q)
            list_of_val.append(dist)
        for value in list_of_val:
            if value > rad:
                decision = False
        
        if decision == True:
            adjacency_mat[i, j] = 1

100%|██████████| 150/150 [02:35<00:00,  1.04s/it]


In [331]:
adjacency_mat = adjacency_mat + adjacency_mat.T
adjacency_mat /= 2
for i in range(n):
    adjacency_mat[i,i] = 0

In [332]:
import networkx as nx
G = nx.from_numpy_array(adjacency_mat)

In [333]:
clusters = list(nx.connected_components(G))
len(clusters)

74

In [334]:
# Trier les clusters par taille décroissante
sorted_clusters = sorted(clusters, key=len, reverse=True)

# Afficher la taille du plus gros cluster
print(f"Taille du plus gros cluster: {len(sorted_clusters[0])}")

# Si vous voulez afficher les tailles des 5 plus gros clusters, par exemple :
top_5_clusters = sorted_clusters[:5]
for i, cluster in enumerate(top_5_clusters, start=1):
    print(f"Taille du cluster {i}: {len(cluster)}")


Taille du plus gros cluster: 44
Taille du cluster 1: 44
Taille du cluster 2: 27
Taille du cluster 3: 8
Taille du cluster 4: 1
Taille du cluster 5: 1


In [339]:
from collections import Counter
misclassified_count = 0

# Parcourir chaque cluster
for cluster in clusters:

    if len(cluster) > 1:
        # Extraire les labels des points dans le cluster
        cluster_labels = [target[i] for i in cluster]
        
        # Trouver le label majoritaire dans le cluster
        majority_label = Counter(cluster_labels).most_common(1)[0][0]
        
        # Compter les misclassifications dans ce cluster
        for i in cluster:
            if target[i] != majority_label:
                misclassified_count += 1

print(f"Nombre total de misclassifications : {misclassified_count}")



Nombre total de misclassifications : 19
